In [5]:
import csv
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

###read csv
#training data
train = []
f1 = open("cancer_train.csv","r")
reader1 = csv.reader(f1)
for line in reader1:
    train.append(line)
f1.close()
#test data
test = []
f2 = open("cancer_test.csv","r")
reader2 = csv.reader(f2)
for line in reader2:
    test.append(line)
f2.close()

###make an array
#training
tda = np.array(train, dtype = np.int64)
#test
tda2 = np.array(test, dtype = np.int64)

###label 생성(index 0 columm)
#train
train_label = []
for i in range(tda.shape[0]):
    train_label.append(tda[i][0])
train_label = np.array(train_label)
train_label = train_label.reshape(tda.shape[0],1)
train_label = train_label.reshape(train_label.shape[0],)
#test
test_label = []
for i in range(tda2.shape[0]):
    test_label.append(tda2[i][0])
test_label = np.array(test_label)
test_label = test_label.reshape(tda2.shape[0],1)
test_label = test_label.reshape(test_label.shape[0],)

###data 생성(index 1~10)
#training
train_data = []
for i in range(tda.shape[0]):
    train_data.append(tda[i][1:10])
train_data = np.array(train_data)
train_data = train_data.reshape(tda.shape[0],train_data.shape[1])

#test
test_data = []
for i in range(tda2.shape[0]):
    test_data.append(tda2[i][1:10])
test_data = np.array(test_data)
test_data = test_data.reshape(tda2.shape[0],test_data.shape[1])

###knn 적용
knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(train_data, train_label)

###predict
#train
pr = knn.predict(train_data)
#test
prtest = knn.predict(test_data)

###accuracy 측정
train_accuracy = np.empty(tda.shape[0])
test_accuracy = np.empty(tda.shape[0])

train_accuracy = knn.score(train_data, train_label)
print("k = 5, knn")
#train_accuracy = accuracy_score(train_label, pr)
print("training accuracy : ", train_accuracy)
#test_accuracy = knn.score(train_data, test_label)
test_accuracy = accuracy_score(test_label,prtest)
print("test accuracy : ", test_accuracy)

########## k 수정
print("\nadjusted 'k' value")
for i in range(1,12):
    print("\nif k = ",i)
    k = i
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(train_data, train_label)
    ###predict
    aptr = knn.predict(train_data)
    aptest = knn.predict(test_data)
    ###accuracy 측정
    aptr_acc = np.empty(tda.shape[0])
    aptest_acc = np.empty(tda.shape[0])
    
    aptr_acc = accuracy_score(train_label, aptr)
    aptest_acc = accuracy_score(test_label, aptest)
    
    print("\nadjust traing acc : ", aptr_acc)
    print("adjust test acc : ", aptest_acc)
    #10개의 feature에 대해 k=3일 때, test acc 가장 높음
    #adjust traing acc :  0.972
    #adjust test acc :  0.9748743718592965

#######feature 개수 수정
#o - 6 /x - 3, 4, 9
#adjust the number of feature (1,2?,3,7,8?,9)
print("\nadjust the number of feature")
feature = [1,2,5,6,7,8]
feature = np.array(feature)
print("the number of feature is ", feature.shape[0])

ad_td = []
for i in range(tda.shape[0]):
    for j in feature:
        ad_td.append(tda[i][j])
ad_td = np.array(ad_td)
ad_td = ad_td.reshape(tda.shape[0],feature.shape[0])
#print(ad_td)

ad_testd = []
for i in range(tda2.shape[0]):
    for j in feature:
        ad_testd.append(tda2[i][j])
ad_testd = np.array(ad_testd)
ad_testd = ad_testd.reshape(tda2.shape[0],feature.shape[0])

for i in range(1,12):
    print("\nif k = ",i)
    k = i
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(ad_td, train_label)
    ###predict
    aptr = knn.predict(ad_td)
    aptest = knn.predict(ad_testd)
    ###accuracy 측정
    aptr_acc = np.empty(tda.shape[0])
    aptest_acc = np.empty(tda2.shape[0])
    
    aptr_acc = accuracy_score(train_label, aptr)
    aptest_acc = accuracy_score(test_label, aptest)
    
    print("\nadjust traing acc : ", aptr_acc)
    print("adjust test acc : ", aptest_acc)
#####(1,3,9) or (3,4,9) feature를 빼면 acc 최대, feature 6개, k = 3 or 5

print("\nmax value")
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(ad_td, train_label)
###predict
aptr = knn.predict(ad_td)
print(aptr)
aptest = knn.predict(ad_testd)
print(aptest)
###accuracy 측정
aptr_acc = np.empty(tda.shape[0])
aptest_acc = np.empty(tda2.shape[0])
    
aptr_acc = accuracy_score(train_label, aptr)
aptest_acc = accuracy_score(test_label, aptest)
    
print("\nadjust traing acc : ", aptr_acc)
print("adjust test acc : ", aptest_acc)

#confusion matrix
#TP FN
#FP TN
#분류기 성능을 평가할때 confusion martix 좋다고 해서 사용해봤습니다.
trcm = confusion_matrix(train_label,aptr)
testcm = confusion_matrix(test_label,aptest)
print("\nconfusion matrix : training data")
print(trcm)
print("\nconfusion matrix : test data")
print(testcm)

#accuracy = (TP+TN)/(TP+FP+TN+FN)
#Precision = TP/(TP+FP)
#Recall = TP/(TP+FN)
#F1 score = 2*((Precision*Recall)/(Precision+Recall)

print("\ntraining data")
train_precision = trcm[0][0]/(trcm[0][0]+trcm[1][0])
train_recall = trcm[0][0]/(trcm[0][0]+trcm[0][1])
train_f1score = 2*((train_precision*train_recall)/(train_precision+train_recall))
print("precision : ", train_precision)
print("recall : ", train_recall)
print("f1 score : ", train_f1score)

print("\ntest data")
test_precision = testcm[0][0]/(testcm[0][0]+testcm[1][0])
test_recall = testcm[0][0]/(testcm[0][0]+testcm[0][1])
test_f1score = 2*((test_precision*test_recall)/(test_precision+test_recall))
print("precision : ", test_precision)
print("recall : ",test_recall)
print("f1 score : ",test_f1score)

k = 5, knn
training accuracy :  0.974
test accuracy :  0.964824120603015

adjusted 'k' value

if k =  1

adjust traing acc :  1.0
adjust test acc :  0.949748743718593

if k =  2

adjust traing acc :  0.976
adjust test acc :  0.9396984924623115

if k =  3

adjust traing acc :  0.972
adjust test acc :  0.9748743718592965

if k =  4

adjust traing acc :  0.972
adjust test acc :  0.9597989949748744

if k =  5

adjust traing acc :  0.974
adjust test acc :  0.964824120603015

if k =  6

adjust traing acc :  0.968
adjust test acc :  0.9597989949748744

if k =  7

adjust traing acc :  0.968
adjust test acc :  0.964824120603015

if k =  8

adjust traing acc :  0.966
adjust test acc :  0.9597989949748744

if k =  9

adjust traing acc :  0.966
adjust test acc :  0.964824120603015

if k =  10

adjust traing acc :  0.966
adjust test acc :  0.9547738693467337

if k =  11

adjust traing acc :  0.968
adjust test acc :  0.9597989949748744

adjust the number of feature
the number of feature is  6

if k 